# Tecnologias de Processamento de dados - 2019/2020

## Phase III - Group 12


|   Student      | Student ID |  Contribution in hours |
|----------------|------------|------------------------|
| Beatriz Lima   |    49377   |                      |
| David Almeida  |    54120   |                      |
|João Castanheira|    55052   |                      |
| Pedro Cotovio  |    55053   |                      |


# 1. OLAP Queries

###  a. Previously defined queries

Check and revise the queries defined in Phase I. Ensure that the queries are meaningful for the processes at hand

### b. For each query type identify dimensions, attributes and measures for each one

### c. SQL queries 
Write the SQL queries and execute them. Make sure the results are understandable


# 2. Bottlenecks

###  a. Most important bottlenecks and performance analysis
Identify the most important bottlenecks and execute performance analysis

### b. Plan for improving performance

Propose a plan for improving performance and implement it

### c. Verify that the proposed modifications have better performance

### d. Discussion

With the full use of the data warehouse what are the expectation for its growth in 1, 5 and 10 years. Identify what approaches would be used to address such an increase in data

# 3. Critical assessment

###  a. Phases comparison
For each of the 3 phases of the project discuss what went right, wrong and what would you do to make it better?


### b. Grade proposal
For each phase propose a grade (1-20). Justify your grade.

